# 02. Structural Pattern Analysis

This notebook investigates structural patterns detected by the analytics pipeline,
such as cycles (mixing), chains (layering), and gathering/scattering.

In [1]:
import sys
import os
# Ensure local utils can be imported
sys.path.append(os.path.abspath('.'))
sys.path.append(os.path.abspath('./notebooks'))
sys.path.append(os.path.abspath('./analytics-pipeline/notebooks'))

from analytics_utils import get_db_client, query_to_df, setup_plotting
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

setup_plotting()

Plotting environment setup complete.


In [2]:
NETWORK = 'torus'
PROCESSING_DATE = '2025-04-18'

## Pattern Summary
Count detected patterns by type.

In [5]:
client = get_db_client(NETWORK)

query = f"""
SELECT 
    pattern_type,
    count() as detected_count,
    avg(toFloat64(evidence_volume_usd)) as avg_volume,
    max(toFloat64(evidence_volume_usd)) as max_volume
FROM analyzers_pattern_detections
WHERE processing_date = toDate('{PROCESSING_DATE}')
GROUP BY pattern_type
ORDER BY detected_count DESC
"""

patterns_df = query_to_df(client, query)
patterns_df

Connecting to ClickHouse: localhost:8125 (DB: analytics_torus)
Connection successful!


,pattern_type,detected_count,avg_volume,max_volume
0,motif_fanout,60,3257.646576,44463.751622
1,motif_fanin,51,4436.827214,100141.860268
2,cycle,3,0.000000,0.000000
3,layering_path,2,94534.681754,100141.859902
4,smurfing_network,2,9114.110716,18228.221432


## High Value Patterns
Investigate specific patterns involving large sums.

In [6]:
query = f"""
SELECT *
FROM analyzers_pattern_detections
WHERE processing_date = toDate('{PROCESSING_DATE}')
AND toFloat64(evidence_volume_usd) > 10000
ORDER BY toFloat64(evidence_volume_usd) DESC
LIMIT 20
"""

top_patterns = query_to_df(client, query)
top_patterns

,window_days,processing_date,pattern_id,pattern_type,pattern_hash,addresses_involved,address_roles,cycle_path,cycle_length,cycle_volume_usd,...,motif_center_address,motif_participant_count,detection_timestamp,pattern_start_time,pattern_end_time,pattern_duration_hours,evidence_transaction_count,evidence_volume_usd,detection_method,_version
0,120,2025-04-18,motif_fanin_d3390d0c0a89da4c,motif_fanin,d3390d0c0a89da4c,[5FbdJn2huDQXi5htanuefe571ZxyiuELRjxQELM7XugTP...,"[center, source, source]",[],0,0E-18,...,5FbdJn2huDQXi5htanuefe571ZxyiuELRjxQELM7XugTPtHr,5,1763668912,0,0,0,2,100141.860268408870000000,motif_detection,1763668912667452
1,120,2025-04-18,layering_path_4f090b84b6996f09,layering_path,4f090b84b6996f09,[5Ef9hhxJwb33DKttcnhM86B6wfbJ3qzvNsJmitQNyNJTR...,"[source, intermediary, destination]",[],0,0E-18,...,,0,1763668912,0,0,0,2,100141.859902143430000000,path_analysis,1763668912667448
2,120,2025-04-18,layering_path_9cde9e7cf4be5413,layering_path,9cde9e7cf4be5413,[5DBmjR3Ycqu1txsE5XewHyzqGSDWEv6V9b4LtAueQYxs4...,"[source, intermediary, destination]",[],0,0E-18,...,,0,1763668912,0,0,0,2,88927.503605950970000000,path_analysis,1763668912667450
3,120,2025-04-18,motif_fanin_fe7ec690e58d1ad5,motif_fanin,fe7ec690e58d1ad5,[5HPLYyHnMPPJvvibY87gkCRJaZrrjfKVUWS1DcjB86dqu...,"[center, source, source]",[],0,0E-18,...,5HPLYyHnMPPJvvibY87gkCRJaZrrjfKVUWS1DcjB86dquy5i,2,1763668912,0,0,0,2,49404.168870743950000000,motif_detection,1763668912667459
4,120,2025-04-18,motif_fanin_d5c3030d1233df1f,motif_fanin,d5c3030d1233df1f,[5FFH4JL8matc5vrEWUHSgVx2ma8cgN9yoiARsDFs5ezSD...,"[center, source, source]",[],0,0E-18,...,5FFH4JL8matc5vrEWUHSgVx2ma8cgN9yoiARsDFs5ezSDPcp,4,1763668912,0,0,0,2,44463.751983669560000000,motif_detection,1763668912667455
5,120,2025-04-18,motif_fanout_289a6833d4c2ef19,motif_fanout,289a6833d4c2ef19,[5FFH4JL8matc5vrEWUHSgVx2ma8cgN9yoiARsDFs5ezSD...,"[center, destination, destination]",[],0,0E-18,...,5FFH4JL8matc5vrEWUHSgVx2ma8cgN9yoiARsDFs5ezSDPcp,4,1763668912,0,0,0,2,44463.751622281416000000,motif_detection,1763668912667456
6,120,2025-04-18,motif_fanout_9d4759560d318753,motif_fanout,9d4759560d318753,[5CF6Ruq6u7DoF6TYxyaFzEuviWi7tjvJtz19XgPS2b7XD...,"[center, destination, destination, destination...",[],0,0E-18,...,5CF6Ruq6u7DoF6TYxyaFzEuviWi7tjvJtz19XgPS2b7XDCFM,32,1763668912,0,0,0,32,33308.403749685300000000,motif_detection,1763668912667462
7,120,2025-04-18,motif_fanout_a2bffbf7b2b255e0,motif_fanout,a2bffbf7b2b255e0,[5DDXwRsgvdfukGZbq2o27n43qyDaAnZ6rsfeckGxnaQ1i...,"[center, destination, destination, destination...",[],0,0E-18,...,5DDXwRsgvdfukGZbq2o27n43qyDaAnZ6rsfeckGxnaQ1ih2D,32,1763668912,0,0,0,32,33211.219205083350000000,motif_detection,1763668912667461
8,120,2025-04-18,motif_fanout_bfda4cb724ef871a,motif_fanout,bfda4cb724ef871a,[5FbdJn2huDQXi5htanuefe571ZxyiuELRjxQELM7XugTP...,"[center, destination, destination, destination]",[],0,0E-18,...,5FbdJn2huDQXi5htanuefe571ZxyiuELRjxQELM7XugTPtHr,5,1763668912,0,0,0,3,27546.638147276575000000,motif_detection,1763668912667454
9,120,2025-04-18,motif_fanout_0acedd7e5c8559a2,motif_fanout,0acedd7e5c8559a2,[5GQgZGdgcyyNDKV9ih6hK11FQ8V1RMcVqqhNvF9mQaHEe...,"[center, destination, destination, destination...",[],0,0E-18,...,5GQgZGdgcyyNDKV9ih6hK11FQ8V1RMcVqqhNvF9mQaHEevqA,4,1763668912,0,0,0,4,24702.084435371977000000,motif_detection,1763668912667460
